In [2]:
from pybalmorel import Balmorel

# Collect results
m = Balmorel('../', gams_system_directory='/appl/gams/47.6.0')
m.collect_results()

Folder analysis not added to scenario as the necessary analysis/model/Balmorel.gms and/or analysis/model/cplex.op4 did not exist
Folder .github not added to scenario as the necessary .github/model/Balmorel.gms and/or .github/model/cplex.op4 did not exist
Folder 225a not added to scenario as the necessary 225a/model/Balmorel.gms and/or 225a/model/cplex.op4 did not exist
Folder logs not added to scenario as the necessary logs/model/Balmorel.gms and/or logs/model/cplex.op4 did not exist
/work3/mberos/Balmorel/base_medtemp/model/MainResults_firsttests.gdx
/work3/mberos/Balmorel/base/model/MainResults_firsttests.gdx
/work3/mberos/Balmorel/N10/model/MainResults_N10.gdx
/work3/mberos/Balmorel/base_medlowtemp/model/MainResults_firsttests.gdx
/work3/mberos/Balmorel/N2/model/MainResults_N2.gdx
/work3/mberos/Balmorel/N70/model/MainResults_N70.gdx
/work3/mberos/Balmorel/N30/model/MainResults_N30.gdx
/work3/mberos/Balmorel/biotransport/model/MainResults_biotransport.gdx
/work3/mberos/Balmorel/N50/m

In [3]:
# Interactive Bar Chart Plotting
m.results.interactive_bar_chart(plot_style='dark')

Result type: balmorel


GridBox(children=(Dropdown(description='Table:', layout=Layout(width='80%'), options=('F_CONS_YCRA', 'F_CONS_Y…

Output()

Stack(children=(GridBox(children=(SelectMultiple(description='Scenario', index=(0,), layout=Layout(height='99%…

Output()

Output()

HBox()

Output()

Output()

Output()